# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
 
    def forward(self, x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

# TODO:define loss function and optimiter
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        # Init
        optimizer.zero_grad()
        # Predict
        y_train_pred = model(images) 
        # Calculate loss
        loss = nn.functional.cross_entropy(y_train_pred, labels)
        # Backpropagation
        loss.backward()
        optimizer.step()

        train_loss += loss
        train_correct += (torch.max(y_train_pred, 1)[1].view(labels.size()).data == labels.data).sum()
    for images0, labels0 in tqdm(test_loader): 
        y_pred = model(images0) 
        # Calculate loss
        loss0 = nn.functional.cross_entropy(y_pred, labels0)
        # Backpropagation
        loss0.backward()
        
        test_loss += loss0
        test_correct += (torch.max(y_pred, 1)[1].view(labels0.size()).data == labels0.data).sum()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    train_loss = train_loss.item() / len(train_loader)
    train_accuracy = train_correct.item() / len(train_loader.dataset)
    test_loss = test_loss.item() / len(test_loader)
    test_accuracy = test_correct.item() / len(test_loader.dataset)
    print("[Epoch: %d] Train Loss: %5.5f Train Accuracy: %5.5f" % (epoch+1, train_loss, train_accuracy))
    print("[Epoch: %d] Test Loss: %5.5f Test Accuracy: %5.5f" % (epoch+1, test_loss, test_accuracy))
    

  0%|          | 2/468 [00:00<00:28, 16.49it/s]

[Epoch: 1] Train Loss: 0.19863 Train Accuracy: 0.93863
[Epoch: 1] Test Loss: 0.06097 Test Accuracy: 0.97930


  0%|          | 2/468 [00:00<00:27, 16.68it/s]

[Epoch: 2] Train Loss: 0.05335 Train Accuracy: 0.98223
[Epoch: 2] Test Loss: 0.03972 Test Accuracy: 0.98520


  0%|          | 2/468 [00:00<00:29, 15.77it/s]

...

[Epoch: 8] Train Loss: 0.00951 Train Accuracy: 0.99530
[Epoch: 8] Test Loss: 0.04532 Test Accuracy: 0.98660


  0%|          | 1/468 [00:00<00:59,  7.88it/s]

[Epoch: 9] Train Loss: 0.01142 Train Accuracy: 0.99468
[Epoch: 9] Test Loss: 0.04133 Test Accuracy: 0.98740


100%|██████████| 78/78 [00:08<00:00,  8.85it/s]

[Epoch: 10] Train Loss: 0.00744 Train Accuracy: 0.99592
[Epoch: 10] Test Loss: 0.04957 Test Accuracy: 0.98430


#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))

Training accuracy: 99.59%
Testing accuracy: 98.43%
